In [1]:
from pathlib import Path
import pandas as pd

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient(host='203.65.72.120')

In [4]:
client.list_database_names()

['Listeria_monocytogenes',
 'Mycobacterium_tuberculosis_variant_bovis',
 'Neisseria_meningitidis',
 'Salmonella_enterica',
 'Vibrio_cholerae',
 'admin',
 'config',
 'local']

In [5]:
database = client['Neisseria_meningitidis']

In [6]:
database.list_collection_names()

['Sample_attribute', 'Description']

In [7]:
collection = database['Sample_attribute']

In [9]:
cursor = collection.find({})

In [25]:
outpath = Path('/media/NGS/SRA_1/NCBI_Neisseria_meningitidis_SRA/Profile')
for document in cursor:
    s = pd.Series(document['profile'], name='allele_id')
    s.index.name = 'locus_id'
    s.to_csv(outpath/(document['NCBIAccession']+'.tsv'), sep='\t')

In [72]:
cursor = collection.find({}, {'profile': False, '_id': False})

In [73]:
df = pd.DataFrame(list(cursor))

In [74]:
df.head()

,Identifier,Resistance gene,ST,gyrA,parC,BioSample,Date,Country,Serovar_ori,Serovar_seq,Void_loci,Host,SourceSeq,Inc_type
0,R15.2199,"blaPSE, dfrA1, erm(42), floR, sul1, sul2, tet(G)",292,p.S83F,p.T57S,SAMN10425279,2015,Taiwan,Albany,Albany,8,Homo sapiens,SRR8189353,NaN
1,R15.2267,"blaPSE, dfrA1, floR, sul1, tet(G)",292,p.S83F,p.T57S,SAMN10425280,2015,Taiwan,Albany,Albany,6,Homo sapiens,SRR8189354,ColpVC
2,R15.2725,"blaPSE, dfrA1, erm(42), floR, mcr-1, sul1, sul...",292,p.S83F,p.T57S,SAMN10425282,2015,Taiwan,Albany,Albany,10,Homo sapiens,SRR8189364,"Col440I, IncI2(Delta)"
3,R16.0431,"blaPSE, dfrA1, erm(42), floR, sul1, sul2, tet(G)",292,p.S83F,p.T57S,SAMN15944976,2015,Taiwan,Albany,Albany,8,Pig,SRR12549528,NaN
4,R16.0556,"blaPSE, dfrA1, erm(42), floR, sul1, sul2, tet(G)",292,p.S83F,p.T57S,SAMN15944967,2015,Taiwan,Albany,Albany,8,Chicken,SRR12549527,NaN


In [75]:
df.to_csv('Salmonella_enterica_track_metadata.csv', index=False)

In [30]:
metadata = pd.read_csv('/home/chen1i6c04/notebook/20200922_S.Goldcoast_track/Metadata.csv', dtype=object)

In [24]:
set(df.columns) - set(metadata.columns)

{'gyrA'}

In [32]:
dirpath = Path('/media/NGS/SRA_1/NCBI_Salmonella_enterica_serovar_Goldcoast_SRA/Profile')

In [35]:
documents = []
for _, row in metadata.iterrows():
    uid = row['Source']
    document = row.dropna().to_dict()
    for i in dirpath.iterdir():
        if i.stem == uid:
            pf = pd.read_csv(i, sep='\t', index_col=0)
            pf = pf.iloc[:, 0].to_dict()
            document['profile'] = pf
            documents.append(document)

In [38]:
collection.insert_many(documents=documents)

In [36]:
from bson import objectid

In [32]:
collection = database['metadata_cols']

In [38]:
collection.find_one()

{'_id': ObjectId('5f640ff05827d7fa536a8a0e'),
 'template': ['Distance (loci)',
  'SourceSeq',
  'BioSample',
  'Identifier',
  'Serovar_ori',
  'Serovar_seq',
  'Date',
  'Country',
  'ST',
  'Host',
  'Inc_type',
  'gyrA',
  'parC',
  'Resistance gene',
  'Void_loci']}

In [29]:
document = {'template': [
    'Distance (loci)', 'SourceSeq', 'BioSample', 'Identifier', 'Serovar_ori', 'Serovar_seq', 'Date',
    'Country', 'ST', 'Host', 'Inc_type', 'gyrA', 'parC', 'Resistance gene', 'Void_loci']}

In [34]:
document

{'template': ['Distance (loci)',
  'SourceSeq',
  'BioSample',
  'Identifier',
  'Serovar_ori',
  'Serovar_seq',
  'Date',
  'Country',
  'ST',
  'Host',
  'Inc_type',
  'gyrA',
  'parC',
  'Resistance gene',
  'Void_loci']}

In [37]:
collection.update_one(
    {'_id': objectid.ObjectId('5f640ff05827d7fa536a8a0e')},
    {'$set': document}
)

In [39]:
client.close()